In [2]:
cd /pfs/work7/workspace/scratch/rk3078-WorkSpace04a/ExperimentalData/insitu/01_croppedData/

/pfs/work7/workspace/scratch/rk3078-WorkSpace04a/ExperimentalData/insitu/01_croppedData


In [8]:
import os
import glob
import numpy as np
from tqdm import tqdm
import h5py
import pandas as pd
from pandas import HDFStore

# path to dataset saved as npy-files
dataset_path_npys="preprocessed/"


# specify path and filename where to save dataset as hdf5 
savepath='hdf5/'
dataset_saveas_filename = 'dataset.hdf5'

if not os.path.exists(savepath):
    os.makedirs(savepath)

In [9]:
# check if hdf5 already exsits. if yes, delete it
if os.path.exists(savepath+ dataset_saveas_filename):
    os.remove(savepath+ dataset_saveas_filename)

In [ ]:
# open h5py file
with h5py.File(savepath+ dataset_saveas_filename, 'w') as f:    
    
    subset_info = sorted(glob.glob(dataset_path_npys  +'*'))
    
    # iterate through the two folders of the subsets "test" and "train"
    for subset_iter in range(len(subset_info)):

        # create group for current subset in the h5py file
        subset_path=subset_info[subset_iter]+"/"
        subsetID=subset_path.split("/")[-2]
        f.create_group(subsetID)
        print(subsetID)
        
        
        # get list of all substrates in current subset and iterate through the substrates
        substrate_info = sorted(glob.glob(subset_path  +'A*'))
        for substrate_iter in tqdm(range(len(substrate_info))):
            
            # create group for current substrate in current subset in the h5py file
            substrate_path=substrate_info[substrate_iter]         
            subID=substrate_path[-3:]
            f.create_group(subsetID+"/"+subID)
            
            # get list of all patches in current substrate and iterate through the patches
            patch_info = sorted(glob.glob(substrate_path  +'/*'))
            for patch_iter in range(len(patch_info)):

                patch_path=patch_info[patch_iter]
                #print(patch_path)
                patchID=patch_path[-6:-4]
                data_to_write=np.load(patch_path).astype('uint16')

                # create h5py dataset with data of current patch in the corresponding group of subset and substrate
                f.create_dataset(subsetID+"/"+subID+"/"+patchID, data= data_to_write)
 
    # create empty groups to save cross-validation split information later on 
    f.create_group("train/cv_splits_5fold")
    f.create_group("train/cv_splits_5fold/fold0")
    f.create_group("train/cv_splits_5fold/fold1")
    f.create_group("train/cv_splits_5fold/fold2")
    f.create_group("train/cv_splits_5fold/fold3")
    f.create_group("train/cv_splits_5fold/fold4")
    
# read-in csv-files and save each in pandas dataframe
labels_test=pd.read_csv(dataset_path_npys+"test/labels.csv")
labels_train=pd.read_csv(dataset_path_npys+"train/labels.csv")

fold0_train=pd.read_csv(dataset_path_npys+"train/cv_splits_5fold/fold0/train.csv")
fold0_val=pd.read_csv(dataset_path_npys+"train/cv_splits_5fold/fold0/val.csv")
fold1_train=pd.read_csv(dataset_path_npys+"train/cv_splits_5fold/fold1/train.csv")
fold1_val=pd.read_csv(dataset_path_npys+"train/cv_splits_5fold/fold1/val.csv")
fold2_train=pd.read_csv(dataset_path_npys+"train/cv_splits_5fold/fold2/train.csv")
fold2_val=pd.read_csv(dataset_path_npys+"train/cv_splits_5fold/fold2/val.csv")
fold3_train=pd.read_csv(dataset_path_npys+"train/cv_splits_5fold/fold3/train.csv")
fold3_val=pd.read_csv(dataset_path_npys+"train/cv_splits_5fold/fold3/val.csv")
fold4_train=pd.read_csv(dataset_path_npys+"train/cv_splits_5fold/fold4/train.csv")
fold4_val=pd.read_csv(dataset_path_npys+"train/cv_splits_5fold/fold4/val.csv")

# use HDFStore function to store pandas dataframes in the corresponding groups which had been created before 
hdf = HDFStore(savepath+ dataset_saveas_filename)
hdf.put('test/labels',labels_test)
hdf.put('train/labels',labels_train)

hdf.put('train/cv_splits_5fold/fold0/train',fold0_train)
hdf.put('train/cv_splits_5fold/fold0/val',fold0_val)
hdf.put('train/cv_splits_5fold/fold1/train',fold1_train)
hdf.put('train/cv_splits_5fold/fold1/val',fold1_val)
hdf.put('train/cv_splits_5fold/fold2/train',fold2_train)
hdf.put('train/cv_splits_5fold/fold2/val',fold2_val)
hdf.put('train/cv_splits_5fold/fold3/train',fold3_train)
hdf.put('train/cv_splits_5fold/fold3/val',fold3_val)
hdf.put('train/cv_splits_5fold/fold4/train',fold4_train)
hdf.put('train/cv_splits_5fold/fold4/val',fold4_val)
hdf.close()

print("done")


test


100%|██████████| 12/12 [00:36<00:00,  3.01s/it]


train


100%|██████████| 26/26 [01:20<00:00,  3.11s/it]


done
